In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.special import softmax
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
import os
import datetime
import tensorflow as tf
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [ ]:
def score_from_model(parent):

    X, y = load_digits(return_X_y=True)
    MinMaxScaler(copy=False).fit_transform(X)
    X = X.reshape(X.shape[0], 8, 8, 1).astype('float32')
    y = to_categorical(y)
    model_1 = models.Sequential()

    model_1.add(layers.InputLayer(input_shape=(8, 8, 1), name='input'))

    model_1.add(layers.Flatten(name='flatten'))

    model_1.add(layers.Dense(sum(parent) * 8, activation='relu', name='hidden1'))

    model_1.add(layers.Dense(10, activation='softmax', name='output'))  

    model_1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    plot_model(model_1, show_shapes=True, show_layer_activations=True)
    val_stop = EarlyStopping(monitor='val_loss', patience=5)
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tb = TensorBoard(logdir, histogram_freq=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
    X_train.shape, y_train.shape, X_test.shape, y_test.shape

    fit_1 = model_1.fit(X_train, y_train, 
                    epochs=10, 
                    # batch_size=32,
                    callbacks=[val_stop, tb],
                    validation_data=(X_test, y_test),verbose=0)
    
    return np.sum(fit_1.history['accuracy'])

In [ ]:
class Gen:

    def __init__(self, n_iter=10, rgen=1):
        #self.rgen = np.random.seed(rgen)
        self.n_iter = n_iter
        self.population = []

    def generate_population(self):
        tmp = []
        for i in range(6*8):
            tmp.append(np.random.choice([0,1]))
        self.population = np.split(np.array(tmp),6)

    def pick_best_parents(self):
        self.scores = []
        for i in self.population:
            self.scores.append(score_from_model(i))
        self.result = softmax(self.scores)
        best_index = np.random.choice(6, 3, replace = False, p = self.result).tolist()
        self.best = []
        for i in best_index:
            self.best.append(self.population[i])
        tmp = np.array(self.scores)
        index = tmp.argsort()[-1:][::-1]
        self.best_best = self.population[index[0]]
        self.best_perc = self.scores[index[0]]


    def cross_over(self):
        self.childs = []
        child_1 = []
        child_2 = []
        c = random.randint(0, 7)
        comb = np.array(list(combinations([0,1,2],2))).tolist()
        for i in comb:
            child_1 = self.best[i[0]]
            child_2 = self.best[i[1]]
            for j in range(c,7):
                child_1[j], child_2[j] = child_2[j], child_1[j]
            self.childs.append(child_1)
            self.childs.append(child_2)

    def mutation(self):
        for i in range(len(self.childs)):
            r = random.randint(0,100)
            if r == 1:
                r_2 = random.randint(0,7)
                if self.childs[i][r_2] == 1:
                    self.childs[i][r_2] = 0
                else:
                    self.childs[i][r_2] = 1

In [ ]:
gen = Gen()
gen.generate_population()
results = []
for i in range(20):
    gen.pick_best_parents()
    print(gen.best_best,gen.best_perc)
    results.append(gen.scores[0])
    gen.cross_over()
    gen.mutation()
    gen.population = gen.childs

In [ ]:
plt.plot(results)